In [1]:
%matplotlib inline

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib, collections, datetime, hashlib, os, re, time, sys
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 读取文件

In [2]:
all_raw = pd.read_json("./hk_data/doq/scan/result_doq.txt",lines=True)
# all_raw = pd.read_json("./week_scan_result/2022-07-08/doq/doq_853_scan/result_doq.txt",lines=True)

location_raw = pd.read_json("./hk_data/doq/location/all_doq_location.txt", lines=True)
# location_raw = pd.read_json("./week_scan_result/2022-07-08/doq/doq_853_location/doq_853_location.txt", lines=True)
location_raw = pd.DataFrame(location_raw,columns = ['query','country','isp','org','as','lat','lon'])
location_raw = location_raw.rename(columns={"query":"ip"})
all_raw = pd.merge(all_raw, location_raw, on=['ip'])
all_raw.drop_duplicates(subset=['ip'], keep='first', inplace=True)


all_raw

,ip,port,tlsVersion,available,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,ocsp_cert_status,ocsp_response_sig,ocsp_stapling_cert_status,ocsp_stapling_response_sig,country,isp,org,as,lat,lon
0,135.125.236.63,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLRE...,DV,True,Let's Encrypt,True,...,Good,True,,False,Germany,OVH SAS,OVH GmbH,AS16276 OVH SAS,50.3986,8.07958
1,172.105.52.109,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZNVE...,DV,True,Let's Encrypt,True,...,Good,True,,False,India,"Linode, LLC",Linode,"AS63949 Linode, LLC",19.0748,72.88560
2,208.72.155.33,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,Good,True,United States,The Constant Company,"Vultr Holdings, LLC","AS20473 The Constant Company, LLC",21.3037,-157.86670
3,23.247.213.11,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,Good,True,United States,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",37.3417,-121.97530
4,23.247.214.224,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,Good,True,France,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",43.3017,5.38000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3431,45.145.225.187,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZMak...,DV,True,Let's Encrypt,True,...,Good,True,,False,Germany,"Oliver Horscht is trading as ""SYNLINQ""",Prepaid Hoster,AS44486 Oliver Horscht is trading as SYNLINQ,50.1109,8.68213
3432,45.90.28.52,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,Good,True,United States,"nextdns, Inc.","nextdns, Inc","AS34939 nextdns, Inc.",39.7528,-75.52990
3433,45.90.30.112,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,Good,True,United States,"nextdns, Inc.","nextdns, Inc","AS34939 nextdns, Inc.",37.3481,-122.01900
3434,45.90.30.67,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,Good,True,United States,"nextdns, Inc.","nextdns, Inc","AS34939 nextdns, Inc.",37.3481,-122.01900


In [3]:
all_raw.keys()

Index(['ip', 'port', 'tlsVersion', 'available', 'connError', 'raw_cert_chain',
       'cert_level', 'certValid', 'caName', 'time_valid', 'cert_lifetime',
       'cert_error', 'ct_tls', 'sct_tls_valid', 'sct_tls_log', 'ct_cert',
       'sct_cert_valid', 'sct_cert_log', 'ct_ocsp', 'sct_ocsp_valid',
       'sct_ocsp_log', 'ct_valid', 'crLs', 'crl_server', 'ocsp', 'ocspServer',
       'ocspStapling', 'ocspMustStaple', 'respect_ms', 'response_flag',
       'crl_cert_status', 'crl_response_sig', 'ocsp_cert_status',
       'ocsp_response_sig', 'ocsp_stapling_cert_status',
       'ocsp_stapling_response_sig', 'country', 'isp', 'org', 'as', 'lat',
       'lon'],
      dtype='object')

In [4]:
all_raw['ct_valid'].value_counts()
print("--------------------------")
all_raw['ct_valid'].value_counts()/len(all_raw)*100

True     3298
False     138
Name: ct_valid, dtype: int64

--------------------------


True     95.983702
False     4.016298
Name: ct_valid, dtype: float64

In [5]:
have_ct_num = len(all_raw[(all_raw['ct_cert']==True) | (all_raw['ct_tls']==True)| (all_raw['ct_ocsp']==True)])
print(have_ct_num)
print(have_ct_num/len(all_raw)*100)

3327
96.82770663562282


In [6]:
def list_convert(arg):
    new_list = []
    for i in arg:
        if type(i) is not list:
            new_list.append(i)
        else:
            new_list.extend(list_convert(i))
    return new_list

In [7]:
cert_log_list = []

cert_ct_raw = all_raw[all_raw['ct_cert']==True]
for index,value in cert_ct_raw["sct_cert_log"].iteritems():
    temp = value.split("#||#")
    cert_log_list.append(temp)
    
temp = list_convert(cert_log_list)
cert_log =pd.Series(temp)
print(cert_log.value_counts())   #null即该域名有cert传递的sct，但是certvalid都为false。或第一个为flase,第二个为true
print("-------------------------------")
print(cert_log.value_counts()/len(temp)*100)
print("-------------------------------")

google_log = 0
for index,value in cert_ct_raw["sct_cert_log"].iteritems():
    temp = value.split("#||#")
    for i in range(len(temp)):
        if "Google" in temp[i]:
            google_log = google_log + 1
            break
print(google_log, google_log/len(cert_ct_raw)*100)
print("-------------------------------")

google_log = 0
all_log = 0
for index,value in cert_log.value_counts().items():
    if "Google" in index:
        google_log = google_log + value
    if index != "":
        all_log = all_log + value
print(google_log, google_log/all_log*100)


Cloudflare 'Nimbus2023' Log    2947
Google 'Xenon2023' log         2828
Let's Encrypt 'Oak2023' log     323
Google 'Argon2023' log          273
Sectigo 'Mammoth' CT log         71
Google 'Xenon2022' log           66
                                 60
Cloudflare 'Nimbus2022' Log      55
Google 'Argon2022' log           52
Let's Encrypt 'Oak2022' log      49
DigiCert Nessie2023 Log          43
Sectigo 'Sabre' CT log            7
DigiCert Nessie2022 Log           4
DigiCert Nessie2024 Log           1
Google 'Argon2024' log            1
dtype: int64
-------------------------------
Cloudflare 'Nimbus2023' Log    43.466077
Google 'Xenon2023' log         41.710914
Let's Encrypt 'Oak2023' log     4.764012
Google 'Argon2023' log          4.026549
Sectigo 'Mammoth' CT log        1.047198
Google 'Xenon2022' log          0.973451
                                0.884956
Cloudflare 'Nimbus2022' Log     0.811209
Google 'Argon2022' log          0.766962
Let's Encrypt 'Oak2022' log     0.722714
DigiC

## CT LOG 

In [8]:
ct_prase = pd.read_json("./hk_data/doq/scan/result_ct_prase.txt",lines=True)
ct_prase

,ip,googleCertLog,appleCertLog
0,135.125.236.63,Let's Encrypt 'Oak2023' log###Cloudflare 'Nimb...,Let's Encrypt 'Oak2023' log###Cloudflare 'Nimb...
1,172.105.52.109,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
2,208.72.155.33,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
3,23.247.213.11,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
4,23.247.214.224,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
...,...,...,...
3353,45.145.225.187,Cloudflare 'Nimbus2023' Log###Google 'Argon202...,Cloudflare 'Nimbus2023' Log###Google 'Argon202...
3354,45.90.28.52,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
3355,45.90.30.112,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
3356,45.90.30.67,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...


In [9]:
all_raw = pd.merge(all_raw,ct_prase,on=['ip'],how='left')
all_raw

,ip,port,tlsVersion,available,connError,raw_cert_chain,cert_level,certValid,caName,time_valid,...,ocsp_stapling_cert_status,ocsp_stapling_response_sig,country,isp,org,as,lat,lon,googleCertLog,appleCertLog
0,135.125.236.63,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZLRE...,DV,True,Let's Encrypt,True,...,,False,Germany,OVH SAS,OVH GmbH,AS16276 OVH SAS,50.3986,8.07958,Let's Encrypt 'Oak2023' log###Cloudflare 'Nimb...,Let's Encrypt 'Oak2023' log###Cloudflare 'Nimb...
1,172.105.52.109,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZNVE...,DV,True,Let's Encrypt,True,...,,False,India,"Linode, LLC",Linode,"AS63949 Linode, LLC",19.0748,72.88560,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
2,208.72.155.33,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,United States,The Constant Company,"Vultr Holdings, LLC","AS20473 The Constant Company, LLC",21.3037,-157.86670,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
3,23.247.213.11,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,United States,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",37.3417,-121.97530,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
4,23.247.214.224,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,France,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",43.3017,5.38000,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3431,45.145.225.187,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUZMak...,DV,True,Let's Encrypt,True,...,,False,Germany,"Oliver Horscht is trading as ""SYNLINQ""",Prepaid Hoster,AS44486 Oliver Horscht is trading as SYNLINQ,50.1109,8.68213,Cloudflare 'Nimbus2023' Log###Google 'Argon202...,Cloudflare 'Nimbus2023' Log###Google 'Argon202...
3432,45.90.28.52,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,United States,"nextdns, Inc.","nextdns, Inc","AS34939 nextdns, Inc.",39.7528,-75.52990,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
3433,45.90.30.112,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,United States,"nextdns, Inc.","nextdns, Inc","AS34939 nextdns, Inc.",37.3481,-122.01900,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
3434,45.90.30.67,853,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJek...,DV,True,ZeroSSL,True,...,Good,True,United States,"nextdns, Inc.","nextdns, Inc","AS34939 nextdns, Inc.",37.3481,-122.01900,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...


In [10]:
all_raw["googleCertLog"] = all_raw["googleCertLog"].fillna("")
all_raw["appleCertLog"] = all_raw["appleCertLog"].fillna("")
all_raw["sct_tls_log"] = all_raw["sct_tls_log"].fillna("")
all_raw["sct_ocsp_log"] = all_raw["sct_ocsp_log"].fillna("")

In [11]:
ALL_ct_log = []
log_file = open("./log_list.json", "r")
log_data = json.loads(log_file.read())
for i in range(len(log_data['operators'])):
    operators = log_data['operators'][i]

    for j in range(len(operators['logs'])):
        logs = operators['logs'][j]
        ALL_ct_log.append(logs['description'])

In [12]:

import datetime
import json
import base64
from OpenSSL import crypto

def get_ct_log(cert_log,tls_log,ocsp_log):
    cert_log_list = cert_log.split("###")
    tls_log_list = tls_log.split("#||#")
    ocsp_log_list = ocsp_log.split("#||#")
    
    result_ct_log = []

    for i in range(len(cert_log_list)):
        log = cert_log_list[i]
#         if log in ALL_ct_log and log not in result_ct_log:
        if log not in result_ct_log:
            result_ct_log.append(log)
    
    for i in range(len(ocsp_log_list)):
        log = ocsp_log_list[i]
#         if log in ALL_ct_log and log not in result_ct_log:
        if log not in result_ct_log:
            result_ct_log.append(log)
    
    for i in range(len(tls_log_list)):
        log = tls_log_list[i]
#         if log in ALL_ct_log and log not in result_ct_log:
        if log not in result_ct_log:
            result_ct_log.append(log)

    return result_ct_log

def check_google_old(log_list):
    google_flag = False
    none_google_flag = False

    for i in range(len(log_list)):
        if 'Google' in log_list[i]:
            google_flag = True
        else:
            none_google_flag = True

    if google_flag and none_google_flag:
        return True
    else:
        return False

def check_google_policy(data):
    raw_cert_chain = data['raw_cert_chain']
    cert_life_time = data['cert_lifetime']
    googleCertLog = data['googleCertLog']
    tls_log = data['sct_tls_log']
    ocsp_log = data['sct_ocsp_log']
    have_ct = data['ct_cert'] or data['ct_ocsp'] or data['ct_tls']

    if raw_cert_chain != "" and have_ct == True and type(googleCertLog) != float :
        leaf_cert = raw_cert_chain.split("###")[0]
        pem = base64.b64decode(leaf_cert)
        cert_body = crypto.load_certificate(crypto.FILETYPE_PEM, pem)  # 从类型为type编码的字符串缓冲区加载证书 (X509)
        notBefore = datetime.datetime.strptime(cert_body.get_notBefore().decode(), "%Y%m%d%H%M%SZ")
        notAfter = datetime.datetime.strptime(cert_body.get_notAfter().decode(), "%Y%m%d%H%M%SZ")

        if data['ip'] == '139.59.158.17':
            print(data['ip'])

        google_time = datetime.datetime.strptime('2022-04-15', "%Y-%m-%d")
        ct_log_list = get_ct_log(googleCertLog,tls_log,ocsp_log)

        if notBefore > google_time:
            if cert_life_time <=180:
                if len(ct_log_list) >= 2:
                    return "success"
                else:
                    return "new;time<=180;log<2"
            else:
                if len(ct_log_list) >= 3:
                    return "success"
                else:
                    return "new;time>180;log<3"
        else:
            google_none_google = check_google_old(ct_log_list)
            if google_none_google:
                if cert_life_time <= 450:
                    if len(ct_log_list) >= 2:
                        return "success"
                    else:
                        return "old;time<=450;log<2"

                elif cert_life_time <= 810:
                    if len(ct_log_list) >= 3:
                        return "success"
                    else:
                        return "old;time<=810;log<3"

                elif cert_life_time <= 1170:
                    if len(ct_log_list) >= 4:
                        return "success"
                    else:
                        return "old;time<=1170;log<4"

                elif cert_life_time > 1170:
                    if len(ct_log_list) >= 5:
                        return "success"
                    else:
                        return "old;time>1170;log<5"

            else:
                return "old;google_none_google"

    else:
        return "no support ct"
    

def check_apple_policy(data):
    raw_cert_chain = data['raw_cert_chain']
    cert_life_time = data['cert_lifetime']
    appleCertLog = data['appleCertLog']
    tls_log = data['sct_tls_log']
    ocsp_log = data['sct_ocsp_log']
    have_ct = data['ct_cert'] or data['ct_ocsp'] or data['ct_tls']
    
    if raw_cert_chain != "" and have_ct == True and type(appleCertLog) != float :
        leaf_cert = raw_cert_chain.split("###")[0]
        pem = base64.b64decode(leaf_cert)
        cert_body = crypto.load_certificate(crypto.FILETYPE_PEM, pem)  # 从类型为type编码的字符串缓冲区加载证书 (X509)
        notBefore = datetime.datetime.strptime(cert_body.get_notBefore().decode(), "%Y%m%d%H%M%SZ")
        notAfter = datetime.datetime.strptime(cert_body.get_notAfter().decode(), "%Y%m%d%H%M%SZ")

        apple_time = datetime.datetime.strptime('2021-04-21', "%Y-%m-%d")
        ct_log_list = get_ct_log(appleCertLog,tls_log,ocsp_log)
        
        if notBefore > apple_time:
            if cert_life_time <=180:
                if len(ct_log_list) >= 2:
                    return "success"
                else:
                    return "new;time<=180;log<2"
            else:
                if len(ct_log_list) >= 3:
                    return "success"
                else:
                    return "new;time>180;log<3"
        else:
            if cert_life_time <=450:
                if len(ct_log_list) >= 2:
                    return "success"
                else:
                    return "new;time<=450;log<2"
            elif cert_life_time <=810:
                if len(ct_log_list) >= 3:
                    return "success"
                else:
                    return "new;time<=810;log<3"
            
            elif cert_life_time <=1170:
                if len(ct_log_list) >= 4:
                    return "success"
                else:
                    return "new;time<=1170;log<4"
                
            else:
                if len(ct_log_list) >= 5:
                    return "success"
                else:
                    return "new;time>1170;log<5"
    else:
        return "no support ct"
        
    
    
def check_subject(data):
    raw_cert_chain = data['raw_cert_chain']

    if raw_cert_chain != "":
        leaf_cert = raw_cert_chain.split("###")[0]
        pem = base64.b64decode(leaf_cert)
        cert_body = crypto.load_certificate(crypto.FILETYPE_PEM, pem)  # 从类型为type编码的字符串缓冲区加载证书 (X509)
        cn =cert_body.get_subject().CN
        org =cert_body.get_subject().O
        ou =cert_body.get_subject().OU
        result = ""

        if cn != None:
            result = result + cn + " "
        
        if org != None:
            result = result + org + " "
        
        if ou != None:
            result = result + ou + " "
        
        result = result.replace(",", " ")
        result = result.replace("@", " ")
        result = result.replace(";", " ")
        result = result.replace("*", "")
        result = result.replace("(", "")
        result = result.replace(")", "")
        return result
    else:
        if type(data['org']) == str and data['org']!= "":
            result = data['org']
            result = result.replace(",", " ")
            result = result.replace("@", " ")
            result = result.replace(";", " ")
            result = result.replace("*", "")
            result = result.replace("(", "")
            result = result.replace(")", "")
            
            return result
        else:
            return "unknown"
        
        

In [13]:
all_raw = all_raw.assign(google_ct_policy=all_raw.apply(lambda x: check_google_policy(x), axis=1))

print(all_raw['google_ct_policy'].value_counts())
print("---------------------------------")
print(all_raw['google_ct_policy'].value_counts()/len(all_raw)*100)

139.59.158.17
success          3327
no support ct     109
Name: google_ct_policy, dtype: int64
---------------------------------
success          96.827707
no support ct     3.172293
Name: google_ct_policy, dtype: float64


In [14]:
all_raw = all_raw.assign(apple_ct_policy=all_raw.apply(lambda x: check_apple_policy(x), axis=1))

print(all_raw['apple_ct_policy'].value_counts())
print("---------------------------------")
print(all_raw['apple_ct_policy'].value_counts()/len(all_raw)*100)

success          3327
no support ct     109
Name: apple_ct_policy, dtype: int64
---------------------------------
success          96.827707
no support ct     3.172293
Name: apple_ct_policy, dtype: float64


In [15]:
temp = all_raw[(all_raw['google_ct_policy'] != 'success')&(all_raw['certValid']==True)]
for index,value in temp.iterrows():
    print(value['ip'])
    print(value['sct_cert_log'])
    print(value['googleCertLog'])
    print(value['google_ct_policy'])
    print("----")




In [17]:
all_raw = all_raw.assign(subject=all_raw.apply(lambda x: check_subject(x), axis=1))

In [18]:
from drain3.drain import Drain, LogCluster
model = Drain(param_str="(.*)")

for entry in list(all_raw['subject']):
    cluster, change_type = model.add_log_message(entry)

print(model.clusters_counter)
template_list = []
for i in range(model.clusters_counter):
    result = str(model.id_to_cluster[i+1]).split(":")[2][1:]
    template_list.append(result)
#     print(model.id_to_cluster[i+1])
# print(template_list)

823


In [19]:
import re
def group_subject(data):
    subject = data['subject']
    org = data['org']
    org = org.replace(",", " ")
    org = org.replace("@", " ")
    org = org.replace(";", " ")
    org = org.replace("*", "")
    org = org.replace("(", "")
    org = org.replace(")", "")
    result = ""
    for i in range(len(template_list)):
        template = template_list[i]
#         print(template)
#         print(subject)
#         print(org)
#         print("--------")
        if subject == template or org == template:
            result = template
            break
        else:
            if len(re.findall(template,subject)) != 0 or len(re.findall(template,org)) != 0:
                result = template
                break
    if result == "": 
        return subject
    else:
        return result


In [21]:
all_raw = all_raw.assign(group=all_raw.apply(lambda x: group_subject(x), axis=1))


In [22]:
# temp = all_raw[(all_raw['group']=="nextdns  Inc") | (all_raw['group']=="dns.nextdns.io")]
temp = all_raw[all_raw['group']=='dns.adguard.com']
temp.country.value_counts()/len(temp)*100

Cyprus    47.058824
Canada    47.058824
Italy      5.882353
Name: country, dtype: float64

## Revoke 

In [23]:
len(all_raw[(all_raw['crLs']==True)|(all_raw['ocsp']==True)|(all_raw['ocspMustStaple']==True)|(all_raw['ocspStapling']==True)])

3340

In [24]:
3340/len(all_raw)*100

97.20605355064028

In [25]:
all_raw['crLs'].value_counts()
print("--------------------------")
all_raw['crLs'].value_counts()/len(all_raw)*100

False    3368
True       68
Name: crLs, dtype: int64

--------------------------


False    98.020955
True      1.979045
Name: crLs, dtype: float64

In [26]:
all_raw[all_raw['crLs']==True].group.value_counts()

CloudFlare Origin Certificate CloudFlare  Inc. CloudFlare Origin CA     10
iyahaha.com                                                              4
Tencent (.*) (.*) Beijing Co. (.*)                                       4
.passcloud.xyz                                                           3
zeroclover.net                                                           2
2099.ml                                                                  2
.leadmon.net                                                             2
yuyz.net                                                                 2
Cloudflare                                                               2
dns.root.me                                                              2
.ipv4dns.com                                                             2
pccoach.ddns.net                                                         1
.thethorsens.org                                                         1
.dns.com                 

In [27]:
all_raw['ocsp_response_sig'].value_counts()
print("--------------------------")
all_raw['ocsp_response_sig'].value_counts()/len(all_raw)*100

True     3250
False     186
Name: ocsp_response_sig, dtype: int64

--------------------------


True     94.586729
False     5.413271
Name: ocsp_response_sig, dtype: float64

In [28]:
all_raw['ocsp'].value_counts()
print("--------------------------")
all_raw['ocsp'].value_counts()/len(all_raw)*100

True     3327
False     109
Name: ocsp, dtype: int64

--------------------------


True     96.827707
False     3.172293
Name: ocsp, dtype: float64

In [29]:
all_raw['ocspStapling'].value_counts()
print("--------------------------")
all_raw['ocspStapling'].value_counts()/len(all_raw)*100

True     2435
False    1001
Name: ocspStapling, dtype: int64

--------------------------


True     70.867288
False    29.132712
Name: ocspStapling, dtype: float64

In [30]:
all_raw['ocspMustStaple'].value_counts()
print("--------------------------")
all_raw['ocspMustStaple'].value_counts()/len(all_raw)*100

True     2444
False     992
Name: ocspMustStaple, dtype: int64

--------------------------


True     71.12922
False    28.87078
Name: ocspMustStaple, dtype: float64

In [31]:
all_raw['respect_ms'].value_counts()
print("--------------------------")
all_raw['respect_ms'].value_counts()/len(all_raw)*100

True     2434
False    1002
Name: respect_ms, dtype: int64

--------------------------


True     70.838184
False    29.161816
Name: respect_ms, dtype: float64

In [32]:
# for index, value in all_raw[(all_raw['ocspStapling']==False)].subject.value_counts().items():
#     print(index,value)
temp = all_raw[(all_raw['ocspStapling']==False)]
len(temp[temp['ocsp']==True])/len(temp)*100

89.11088911088912

In [33]:
all_raw[(all_raw['crLs']==True)].caName.value_counts()

Google Trust Services LLC      19
GlobalSign nv-sa               15
CloudFlare, Inc.               11
DigiCert, Inc.                  5
GoGetSSL                        5
Buypass AS-983163327            4
TrustCor Systems S. de R.L.     3
Cloudflare, Inc.                2
Gandi                           1
DigiCert Inc                    1
home.pl S.A.                    1
Root Networks, LLC              1
Name: caName, dtype: int64